In [ ]:
%%writefile my_model.py

# fairing 패키지에 python 파일을 추가하기

import os
import datetime
import tensorflow as tf    

class MyModel(object):
    def train(self):
        mnist = tf.keras.datasets.fashion_mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
        print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)

        x_train, x_test = x_train / 255.0, x_test / 255.0

        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        model.compile(optimizer='sgd',
                        loss='sparse_categorical_crossentropy',
                        metrics=['acc'])


        date_folder = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
        if os.getenv('FAIRING_RUNTIME', None) is None:
            log_dir = "log/fit/" + date_folder
        else:
            log_dir = "/notebook/log/fit/" + date_folder 

        print(f"tensorboard log dir : {log_dir}")

        tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                        histogram_freq=1)
        model.fit(x_train, y_train,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    epochs=10,
                    callbacks=[tensorboard_cb])
        return model

In [ ]:
# fairing_outputmap.ipynb
import os
from my_model import MyModel
from kubeflow import fairing
from kubeflow.fairing.kubernetes.utils import mounting_pvc

def train():
    my_model = MyModel()
    my_model.train()   
    

# output_map 에 key[현재경로의 파일이름]:value[컨테이너 안의 파일경로] 형태로 넣어줍니다.
output_map =  {
    "my_model.py": "/app/my_model.py"
}         

# faring preprocessor 사용하기
# preprocessor에서 ouput_map을 넣음으로써 fairing 패키지 안에 my_model.py가 들어가게 됩니다.
fairing.config.set_preprocessor("function", 
                                function_obj=train, # fairing으로 실행시킬 함수 
                                output_map=output_map) # 넣을 파일의 딕셔너리 자료형

# DOCKERHUB 사용시
DOCKER_REGISTRY = 'YOURID' # 도커허브아이디

# CAP HARBOR REGISTRY 사용시
# DOCKER_REGISTRY = 'cap.dudaji.com:31480/프로젝트 이름'

fairing.config.set_builder(
    'append',
    image_name='fashion-mnist-job', 
    base_image='dudaji/cap-jupyterlab:tf2.0-cpu',
    registry=DOCKER_REGISTRY, 
    push=True)

notebook_volume = mounting_pvc(pvc_name="workspace-handson", 
                                pvc_mount_path="/notebook")

fairing.config.set_deployer('job',
                            pod_spec_mutators=[notebook_volume],
                            cleanup=True) # 잡을 실행후 완료시 잡을 삭제할지의 여부를 결정

if __name__ == '__main__':
    fairing.config.run()